## Time series prediction

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from utils import MACD, RSI, Drawdown_current, volatility_rolling, extract, Close_price, log_return, WFCV, feature_engineering, main
import statsmodels.api as sm

import hydra
from omegaconf import DictConfig, OmegaConf

In [2]:
header_df = pd.read_csv('230216_returns.csv', nrows=0)
tickers = header_df.columns[1:].tolist()

print(f"Actifs trouvés : {tickers}")
print(f"Nombre d'actifs : {len(tickers)}")

Actifs trouvés : ['ADBF Index', 'ADCM Index', 'ADCT Index', 'ADEG Index', 'ADHC Index', 'ADII Index', 'ADSMI Index', 'AEDEUR Curncy', 'AEDJPY Curncy', 'AEDUSD Curncy', 'AHDY Comdty', 'AON Index', 'ARSEUR Curncy', 'ARSJPY Curncy', 'ARSUSD Curncy', 'AS504 Index', 'AS509 Index', 'AS51 Index', 'AS519 Index', 'AS51COND Index', 'AS51CONS Index', 'AS51ENGY Index', 'AS51FIN Index', 'AS51HC Index', 'AS51HOTR Index', 'AS51INDU Index', 'AS51IT Index', 'AS51MATL Index', 'AS51REAL Index', 'AS51UTIX Index', 'AS521 Index', 'ASE Index', 'ASX Index', 'M1CXNNA Index', 'AUDEUR Curncy', 'AUDJPY Curncy', 'AUDUSD Curncy', 'BAHYOAS Index', 'BASPTDSP Index', 'BCEA1T Index', 'BCEB1T Index', 'BCEN1T Index', 'BCES1T Index', 'BCOMGR Index', 'BCOMOJ Index', 'BCOMRB Index', 'BCOMSO Index', 'BDTEUR Curncy', 'BDTJPY Curncy', 'BDTUSD Curncy', 'BEATG1 Index', 'BEATG2 Index', 'BEATG3 Index', 'BEATG4 Index', 'BEATG5 Index', 'BEBGTRUU Index', 'BEENRG Index', 'BEGYP Index', 'BEIC4T Index', 'BEID6T Index', 'BEINP Index', 'B

In [ ]:
all_results = []
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
used_tickers = ['GC1 Comdty', 'SI1 Comdty', 'HG1 Comdty',]


for name in tickers:
    res = main(name, model, plot=False)
    all_results.append(res)

df_final = pd.DataFrame(all_results)
df_final.to_csv('Resultats/resultats_test_debug.csv', index=False)

--- Loading dataset for the first time ---


/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [10]:
df  = pd.read_csv('230216_returns.csv')
df

,Date,ADBF Index,ADCM Index,ADCT Index,ADEG Index,ADHC Index,ADII Index,ADSMI Index,AEDEUR Curncy,AEDJPY Curncy,...,WIGFOOD Index,XU100 Index,XUHIZ Index,XUMAL Index,XUSIN Index,XUTEK Index,YW1 Comdty,ZAREUR Curncy,ZARJPY Curncy,ZARUSD Curncy
0,2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005267,0.017349,...,-0.018581,0.151423,0.145900,0.154006,0.136939,NaN,-0.009249,0.001884,0.024162,0.006806
1,2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002269,0.009011,...,-0.051538,-0.033120,0.009194,-0.056499,0.013088,NaN,-0.005834,0.001254,0.011796,0.003593
2,2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000758,0.010369,...,0.034560,-0.043232,0.067112,-0.079351,0.024179,NaN,NaN,-0.002505,0.010493,-0.001699
3,2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003414,0.001141,...,0.041828,-0.022407,-0.003132,-0.018369,-0.026677,NaN,-0.005869,0.006905,0.003461,0.003465
4,2000-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003781,-0.001803,...,0.030046,NaN,NaN,NaN,NaN,NaN,-0.002361,0.001870,-0.006324,-0.002060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5998,2022-12-30,NaN,NaN,NaN,NaN,NaN,NaN,-0.005443,-0.003917,-0.014274,...,0.015670,0.005887,0.001801,0.004680,0.009314,0.037528,0.006700,-0.009042,-0.017636,-0.004343
5999,2023-01-02,NaN,NaN,NaN,NaN,NaN,NaN,0.004077,0.003932,-0.003356,...,0.029938,0.027580,0.027591,0.026521,0.020818,0.028264,NaN,0.007299,-0.005737,0.002436
6000,2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,0.003423,0.010967,0.002254,...,0.003091,-0.006100,-0.004944,-0.013701,-0.014111,0.026584,-0.006441,0.009058,0.005196,-0.001020
6001,2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.001424,-0.007361,-0.007535,...,0.000374,-0.011588,-0.016006,-0.000390,-0.015522,-0.034121,-0.012100,0.005386,0.003572,0.011637


In [32]:
colonnes_vides = df.columns[df.isna().all()].tolist()

print("Colonnes à supprimer :", colonnes_vides)

Colonnes à supprimer : []


In [ ]:
df_final = pd.DataFrame(all_results)
df_final.to_csv('Resultats/resultats_test_debug.csv', index=False)

In [ ]:
liste_remplissage = df.count().sort_values(ascending=True)

print("Nombre d'éléments non-nuls par colonne :")
print(liste_remplissage[:20]) # Je pense qu'il faut virer les colonnes à moins de minimum 250 élements sinon ça fait couiller la WFCV + pas pertinent comme résultats avec si peu de données



Nombre d'éléments non-nuls par colonne :
KOCRD Index        53
CABROVER Index     63
NOBRON Index       64
EB03/3M Index     124
BZSTSETA Index    131
EGDIMW Index      259
MXNZ0EN Index     294
MXNO0UT Index     318
MXGR0HC Index     339
EB10/ON Index     355
KLIB1M Index      364
MXTR0HC Index     390
MXAT0CS Index     468
MXAT0CD Index     480
OBX03 Index       497
MADXUTIL Index    499
MXRU0CD Index     536
MXAR0IN Index     543
DFMATERL Index    562
MXRU0HC Index     585
dtype: int64
